# Task 4: Агрегации

На предыдущих этапах вы занимались просто подготовкой данных. Вы не получили никакой аналитической информации на основе имеющихся данных. Пришло время приступить к анализу. Помните, что, согласно нашему предположению, пользователи приступили бы к работе над лабораторными заданиями раньше, если бы они просмотрели ленту новостей? Это означает, что ключевой метрикой для нас является период времени (delta) между датой начала работы пользователя над лабораторным заданием (первого коммита) и сроком сдачи лабораторного задания. Мы будем смотреть на то, меняется ли она в зависимости от просмотра страницы.

## 1. Создайте соединение с базой данных с помощью библиотеки `sqlite3`.

In [1]:
import pandas as pd
import sqlite3 as s
connection = s.connect('C:/Users/зщ/Downloads/DS_project11-0-develop/src/data/checking-logs.sqlite') 

## 2. Получите схему таблицы `test`.

In [2]:
shema_test = pd.io.sql.read_sql( con = connection,
                            sql = 'PRAGMA table_info(test);' )
shema_test

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,uid,TEXT,0,None,0
2,2,labname,TEXT,0,None,0
3,3,first_commit_ts,TIMESTAMP,0,None,0
4,4,first_view_ts,TIMESTAMP,0,None,0


## 3. Получите только первые 10 строк таблицы `test`, чтобы проверить, как она выглядит.

In [3]:
pd.io.sql.read_sql( con = connection,
                    sql = 'SELECT * FROM test limit 10;' )

,index,uid,labname,first_commit_ts,first_view_ts
0,3,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
1,4,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
2,7,user_30,laba04s,2020-04-18 14:51:37.498399,2020-04-17 22:46:26.785035
3,8,user_14,laba04,2020-04-18 15:14:00.312338,2020-04-18 10:53:52.623447
4,11,user_14,laba04s,2020-04-18 22:30:30.247628,2020-04-18 10:53:52.623447
5,18,user_19,laba04,2020-04-20 19:05:01.297780,2020-04-21 20:30:38.034966
6,19,user_25,laba04,2020-04-20 19:16:50.673054,2020-05-09 23:54:54.260791
7,20,user_21,laba04,2020-04-21 17:48:00.487806,2020-04-22 22:40:36.824081
8,21,user_30,project1,2020-04-22 12:36:24.053518,2020-04-17 22:46:26.785035
9,23,user_21,laba04s,2020-04-22 20:09:21.857747,2020-04-22 22:40:36.824081


## 4. Найдите среди всех пользователей минимальное значение этой самой дельты (периода времени от даты первого коммита пользователя до срока сдачи соответствующего лабораторного задания), используя только один запрос

* Для этого сджойните данные своей таблицы с данными таблицы `deadlines`
* Период времени должен отображаться в часах.
* Не учитывайте лабораторное задание `project1` с более длительным сроком сдачи, поскольку оно будет выделяться на фоне других.
* Сохраните значение в датафрейме `df_min` с соответствующим `uid`.

In [4]:
shema_deadlines = pd.io.sql.read_sql( con = connection,
                            sql = 'PRAGMA table_info(deadlines);' )
shema_deadlines

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,labs,TEXT,0,None,0
2,2,deadlines,INTEGER,0,None,0


In [5]:
pd.io.sql.read_sql( con = connection,
                    sql = 'SELECT * FROM test limit 10;' )

,index,uid,labname,first_commit_ts,first_view_ts
0,3,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
1,4,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
2,7,user_30,laba04s,2020-04-18 14:51:37.498399,2020-04-17 22:46:26.785035
3,8,user_14,laba04,2020-04-18 15:14:00.312338,2020-04-18 10:53:52.623447
4,11,user_14,laba04s,2020-04-18 22:30:30.247628,2020-04-18 10:53:52.623447
5,18,user_19,laba04,2020-04-20 19:05:01.297780,2020-04-21 20:30:38.034966
6,19,user_25,laba04,2020-04-20 19:16:50.673054,2020-05-09 23:54:54.260791
7,20,user_21,laba04,2020-04-21 17:48:00.487806,2020-04-22 22:40:36.824081
8,21,user_30,project1,2020-04-22 12:36:24.053518,2020-04-17 22:46:26.785035
9,23,user_21,laba04s,2020-04-22 20:09:21.857747,2020-04-22 22:40:36.824081


In [6]:
sql = (
           '''
           SELECT   uid, MIN(strftime("%s", first_commit_ts) - deadlines.deadlines)/3600 AS delta
           FROM      test
           INNER JOIN deadlines ON test.labname = deadlines.labs
           WHERE labname <> 'project1'      
           '''
           )

In [7]:
df_min = pd.io.sql.read_sql(sql, connection)
df_min

,uid,delta
0,user_30,-202


## 5. Выполните те же самые операции для максимального значения дельты, используя только один запрос. Название итогового датафрейма — `df_max`.

In [8]:
sql2 = (
           '''
           SELECT   uid, MAX(strftime("%s", first_commit_ts) - deadlines.deadlines)/3600 AS delta
           FROM      test
           INNER JOIN deadlines ON test.labname = deadlines.labs
           WHERE labname <> 'project1'      
           '''
           )

In [9]:
df_max = pd.io.sql.read_sql(sql2, connection)
df_max

,uid,delta
0,user_25,-2


## 6. Выполните те же самые операции для среднего значения дельты, используя только один запрос. На этот раз ваш итоговый датафрейм не должен включать столбец `uid`; имя датафрейма — `df_avg`.

In [10]:
sql3 = (
           '''
           SELECT   AVG((strftime("%s", first_commit_ts) - deadlines.deadlines)/3600) AS delta
           FROM      test
           INNER JOIN deadlines ON test.labname = deadlines.labs
           WHERE labname <> 'project1'      
           '''
           )

In [11]:
df_avg = pd.io.sql.read_sql(sql3, connection)
df_avg

,delta
0,-89.125


## 7. Мы хотим проверить предположение о том, что у пользователей, посетивших ленту новостей всего несколько раз, меньше период времени между датой первого коммита и сроком сдачи лабораторного задания. Для этого вам необходимо рассчитать коэффициент корреляции между количеством просмотров страниц и разницей между датами.

* Используя только один запрос, создайте таблицу со столбцами: `uid`,  `avg_diff`, `pageviews`.
* `uid` — это `uid`, существующие в таблице `test`.
* `avg_diff` — среднее значение дельты (периода времени между датой первого коммита и сроком сдачи лабораторного задания) для каждого пользователя.
* `pageviews` — количество посещений ленты новостей одним пользователем.
* Не учитывайте лабораторное задание `project1`.
* Сохраните результаты в датафрейме `views_diff`.
* Используйте метод `corr()` библиотеки Pandas для вычисления коэффициента корреляции между количеством просмотров и значением дельты.

In [12]:
views_diff = pd.io.sql.read_sql(
             '''
             WITH a AS 
             ( SELECT DISTINCT uid, count(datetime) AS cnt_dt FROM pageviews GROUP BY uid )
             SELECT test.uid, 
                    AVG(( strftime( "%s", test.first_commit_ts ) - deadlines.deadlines ) / 3600 ) AS avg_diff,
                    a.cnt_dt AS pageviews               
             FROM test
             LEFT OUTER JOIN deadlines on test.labname = deadlines.labs AND test.labname != 'project1'
             LEFT OUTER JOIN a ON test.uid = a.uid
             GROUP BY test.uid
             ;''', con=connection , parse_dates = ['deadlines.deadlines', 'test.first_commit_ts'] )
 
views_diff

,uid,avg_diff,pageviews
0,user_1,-64.400000,28
1,user_10,-74.800000,89
2,user_14,-159.000000,143
3,user_17,-61.600000,47
4,user_18,-5.666667,3
5,user_19,-98.750000,16
6,user_21,-95.500000,10
7,user_25,-92.600000,179
8,user_28,-86.400000,149
9,user_3,-105.400000,317


In [13]:
views_diff.corr()

,avg_diff,pageviews
avg_diff,1.000000,-0.279736
pageviews,-0.279736,1.000000


In [14]:
df_min #проверка

,uid,delta
0,user_30,-202


In [15]:
df_max #проверка

,uid,delta
0,user_25,-2


In [16]:
df_avg #проверка

,delta
0,-89.125


In [17]:
views_diff.corr() #проверка

,avg_diff,pageviews
avg_diff,1.000000,-0.279736
pageviews,-0.279736,1.000000


## 8. Закройте соединение.

In [18]:
connection.close()